In [3]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


---
# Project 2 - Part 2
For Part 2, attempt to fine-tune the question-answering with your questions and answers intended to offer greater resolution than the answers in Part 1.
    To do this, you will need to generate your own custom SQuAD-compatible QA dataset using the sections of your choice, the questions, and the answers which you will add to the squad dataset downloadable from Hugging Face, [which is described in this article](https://huggingface.co/transformers/v3.2.0/custom_datasets.html).  Note: custom training can take a very long time on Google Colab, so be prepared for that.

In [4]:
import os
from pprint import pprint
from transformers import pipeline
from transformers import DistilBertTokenizerFast, TFDistilBertForQuestionAnswering

from custom_squad import *  # creating and loading in custom squad questions and answers
from question_answering import *  # automating the question answering process

2022-10-31 10:14:19.870887: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 10:14:19.949031: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-31 10:14:19.983571: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-31 10:14:20.442490: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

---
# Load in Finetuned Distilbert

In [5]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForQuestionAnswering.from_pretrained('model-weights')
generator = pipeline(task="question-answering", model=model, tokenizer=tokenizer)


2022-10-31 10:14:22.748114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 10:14:22.818577: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 10:14:22.818672: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 10:14:22.819055: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

---
# Load in Custom Questions

In [7]:
samples = create_samples()


In [8]:
# peek at a samples
print(f'Num Samples: {len(samples)}')
print(f'Sample Keys: {samples[0].keys()}')


Num Samples: 25
Sample Keys: dict_keys(['id', 'title', 'context', 'question', 'answers'])


In [9]:
print(f'sample[0]:')
pprint(samples[0])


sample[0]:
{'answers': {'answer_start': [608], 'text': ['practitioner']},
 'context': 'Mr. Sherlock Holmes, who was usually very late in the mornings, '
            'save upon those not infrequent occasions when he was up all '
            'night, was seated at the breakfast table. I stood upon the '
            'hearth-rug and picked up the stick which our visitor had left '
            'behind him the night before. It was a fine, thick piece of wood, '
            'bulbous-headed, of the sort which is known as a \\"Penang '
            'lawyer.\\" Just under the head was a broad silver band nearly an '
            'inch across. \\"To James Mortimer, M.R.C.S., from his friends of '
            'the C.C.H.,\\" was engraved upon it, with the date \\"1884.\\" It '
            'was just such a stick as the old-fashioned family practitioner '
            'used to carry—dignified, solid, and reassuring. \\"Well, Watson, '
            'what do you make of it?\\" Holmes was sitting with his b

---
# Test QA

In [10]:
# Test pipeline generator
_ = qa(generator, samples[0], verbose=True)


QUESTION: "What is the guests occupation?"
GROUND TRUTH: "practitioner"
PREDICTED: "country practitioner"
SCORE: "0.5441751480102539"


---
# Finetuned Question vs Answers With 500 Word Contexts

In [11]:
# Run all of the samples through the generator
predicted = list(map(lambda s: qa(generator, s, verbose=False), samples))

# Package the sample with the predicted
questions_answers = list(zip(samples, predicted))


In [12]:
n = len(questions_answers)

# print predictions to console for quick inspection
for i, answer in enumerate(questions_answers):
    print_answer(i, questions_answers[i], n=n)

# write predictions to a file for later
with open(os.path.join('results', 'fine-tuned-500-word-context.txt'), 'w') as f:
    for i, answer in enumerate(questions_answers):
        print_answer(i, questions_answers[i], n=n, file=f)


****************************************************************************************************

QUESTION #0:
"What is the guests occupation?"

GROUND TRUTH ANSWER:
"practitioner"

PREDICTED ANSWER (score=0.5441751480102539):
"country practitioner"

****************************************************************************************************

QUESTION #1:
"How did Sherlock Holmes see Watson?"

GROUND TRUTH ANSWER:
"well-polished, silver-plated coffee-pot"

PREDICTED ANSWER (score=0.08912938088178635):
"an examination of it"

****************************************************************************************************

QUESTION #2:
"What did Holmes and Watson inspect?"

GROUND TRUTH ANSWER:
"visitor's stick"

PREDICTED ANSWER (score=0.09732741862535477):
"our visitor's stick"

****************************************************************************************************

QUESTION #3:
"Where was Sherlock Holmes sitting?"

GROUND TRUTH ANSWER:
"at the breakfast ta

---
# Finetuned Question vs Answers With Whole Book As Context

In [13]:
# read in whole book
context = clean_context(os.path.join('text', 'the-hound-of-the-baskervilles.txt'))

# change samples contexts to full book
samples_large_context = samples.copy()
for i in range(len(samples_large_context)):
    samples_large_context[i]['context'] = context


In [11]:
# Run all of the samples through the generator
predicted = list(map(lambda s: qa(generator, s, verbose=False), samples_large_context))

# Package the sample with the predicted
questions_answers = list(zip(samples_large_context, predicted))


In [13]:
n = len(questions_answers)

# print predictions to console for quick inspection
for i, answer in enumerate(questions_answers):
    print_answer(i, questions_answers[i], n=n)

# write predictions to a file for later
with open(os.path.join('results', 'fine-tuned-book-context.txt'), 'w') as f:
    for i, answer in enumerate(questions_answers):
        print_answer(i, questions_answers[i], n=n, file=f)

****************************************************************************************************

QUESTION #0:
"What is the guests occupation?"

GROUND TRUTH ANSWER:
"practitioner"

PREDICTED ANSWER (score=0.8862924575805664):
"a second cab"

****************************************************************************************************

QUESTION #1:
"How did Sherlock Holmes see Watson?"

GROUND TRUTH ANSWER:
"well-polished, silver-plated coffee-pot"

PREDICTED ANSWER (score=0.576500654220581):
"murder"

****************************************************************************************************

QUESTION #2:
"What did Holmes and Watson inspect?"

GROUND TRUTH ANSWER:
"visitor's stick"

PREDICTED ANSWER (score=0.9183873534202576):
"the Nets"

****************************************************************************************************

QUESTION #3:
"Where was Sherlock Holmes sitting?"

GROUND TRUTH ANSWER:
"at the breakfast table"

PREDICTED ANSWER (score=0.8491

In [14]:
generator(question='Who is the protagonist?', context=context)


{'score': 0.9347644448280334,
 'start': 223321,
 'end': 223331,
 'answer': 'Dr. Watson'}

In [15]:
generator(question='Who is the perpetrator?', context=context)


{'score': 0.872819721698761,
 'start': 77842,
 'end': 77851,
 'answer': 'Barrymore'}

In [16]:
generator(question='What is the setting of the crime?', context=context)


{'score': 0.7306256294250488,
 'start': 157214,
 'end': 157228,
 'answer': 'Merripit House'}

In [17]:
generator(question='What is the evidence against the perpetrator?', context=context)


{'score': 0.6493549346923828,
 'start': 49655,
 'end': 49674,
 'answer': 'he avoided the moor'}